In [1]:
!pip install transformers
import transformers as ppb 
import re
from bs4 import BeautifulSoup
import numpy as np
import torch
import pandas as pd
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel,ppb.BertTokenizer, 'bert-base-cased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t), df.iloc[:,1]))

In [6]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))

In [7]:
b=np.zeros((np.shape(df)[0]))
for i in range(0,np.shape(df)[0]):
  b[i]=np.shape(train_tokens_ids[i])[0]

In [8]:
pad=np.zeros((np.shape(df)[0],int(np.max(b))))
for i in range(0,np.shape(df)[0]):
  a=train_tokens_ids[i]
  for j in range(0,np.shape(a)[0]):
    pad[i,j]=a[j]

In [9]:
pad1=pad[:,0:150]       #pad1=pad[:,0:300]
input_ids = torch.tensor(np.array(pad1))        
with torch.no_grad():
  last_hidden_states = model(input_ids.long()) 
features = last_hidden_states[0][:,0,:].numpy()

In [10]:
features

array([[ 0.25158158,  0.06544146,  0.03632621, ...,  0.43549487,
        -0.04873843,  0.02371828],
       [ 0.30734295,  0.0462268 ,  0.07097965, ...,  0.33745652,
         0.15762296,  0.42273402],
       [ 0.33030477,  0.21474199,  0.5172327 , ..., -0.03585207,
        -0.29864776,  0.32629243],
       ...,
       [ 0.17179191, -0.04870545, -0.0077695 , ...,  0.08644232,
         0.2850142 ,  0.40955842],
       [ 0.28869396, -0.01770288, -0.14143673, ...,  0.10606311,
         0.2667826 ,  0.41769192],
       [ 0.19270162, -0.1402607 , -0.3939941 , ..., -0.12173662,
         0.10633995,  0.56253946]], dtype=float32)

In [11]:
len(features)

1744

In [12]:
DF = pd.DataFrame(features)
from google.colab import files
DF.to_csv('gpt2_algebra.csv') 
files.download('gpt2_algebra.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>